In [1]:
#This script implements pedestrian traffic forecast using Facebook's Prophet

#The inputs required are: date, hour, intersection ID, frecast horizon

#Requirements: 'PedestriansData.sav' or ('cycped_vol.csv' and 'weather.csv')

In [2]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout, Button, Layout
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import warnings
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from fbprophet import Prophet
#import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
#import statsmodels.api as sm
import matplotlib
from DataStructuring import get_data

In [3]:
grid = GridspecLayout(1, 2, height='300px', grid_gap="10px")

In [4]:
file = open("pedestrianTraffic.jpg", "rb")
image = file.read()
pedsImage=widgets.Image(
    value=image,
    format='jpg',
    width=1200,
    height=10,
)
grid[0,:]= pedsImage

In [5]:
display(grid)

GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xfe\…

In [6]:
#Some plot settings
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
plt.close('all')
pd.plotting.register_matplotlib_converters()

In [7]:
#This cell requires either the files: ('PedestriansData.sav' and 'CyclistsData.sav') or ('cycped_vol.csv' and 'weather.csv')
    
filename = 'PedestriansData.sav'
Pedestrians = pickle.load(open(filename, 'rb'))

filename = 'CyclistsData.sav'
Cyclists = pickle.load(open(filename, 'rb'))

filename = 'Locations.sav'
Locations = pickle.load(open(filename, 'rb'))

#Pedestrians, Cyclists, Locations = get_data()

In [8]:
def get_param(ForecastDate,StartHour,forecastHorizon,InterSection,RoadUser):
    
    FirstDate= ForecastDate.value

    FirstTime=int(StartHour.value);

    Horizon=forecastHorizon.value

    testStart = datetime(FirstDate.year, FirstDate.month, FirstDate.day)
    testStart = testStart + pd.Timedelta(hours=FirstTime);
    testEnd = testStart + pd.Timedelta(hours=Horizon-1);
    
    if InterSection.value == 'Adelaide-Bay':
        IntID= 0;
    elif InterSection.value == 'Adelaide-Jarvis':
        IntID= 1;
    elif InterSection.value == 'Front-Bathurst':
        IntID= 2;
    elif InterSection.value == 'King-Portland':
        IntID= 3;
    elif InterSection.value == 'King-Peter':
        IntID= 4;
    elif InterSection.value == 'King-Bay':
        IntID= 5;
    elif InterSection.value == 'King-Yonge':
        IntID= 6;
    elif InterSection.value == 'King-Church':
        IntID= 7;
    elif InterSection.value == 'King-Jarvis':
        IntID= 8;
    elif InterSection.value == 'Queen-Bathurst':
        IntID= 14;
        print('We don\'t have historical data for this intersection, but we will give you an estimate...')
    elif InterSection.value == 'Richmond-Spadina':
        IntID= 9;
    elif InterSection.value == 'Richmond-Bay':
        IntID= 10;
    elif InterSection.value == 'Wellington-Blue_Jays':
        IntID= 11;
    elif InterSection.value == 'Wellington-Bay':
        IntID= 12;
    elif InterSection.value == 'King-John':
        IntID= 13;
        
    if RoadUser.value == 'Pedestrians':
        RoadUserID = 1;
    elif RoadUser.value == 'Cyclists':
        RoadUserID = 2;
        
    return RoadUserID, InterSection.value, IntID, testStart, testEnd, Horizon

In [9]:
def get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon):
    #print("This may take a while...\n\n")
    
    if RoadUserID == 1:
        X = Pedestrians
    elif RoadUserID == 2:
        X = Cyclists

    TrainSize = 24*7*4;
    trainStart = testStart - pd.Timedelta(hours = TrainSize)
    trainEnd = testStart - pd.Timedelta(hours = 1)

    Peds = X[IntID];
    trainVolume = Peds[trainStart: trainEnd]['volume']

    testVolume = Peds[testStart: testEnd]['volume'];
    weatherForecast = Peds[trainStart: testEnd][['tempC','humidity']];

    pInput = Peds[trainStart: trainEnd][['dateTimeIdx','volume',
                 'tempC','humidity']]
    pInput = pInput.rename(columns={"dateTimeIdx": "ds", "volume": "y"})

    #Using Prophet
    m = Prophet(yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=True);
    m.add_regressor('tempC')
    m.add_regressor('humidity')
    m.fit(pInput)
    future = m.make_future_dataframe(periods = Horizon, freq = 'h');
    future['tempC'] = weatherForecast['tempC'].values
    future['humidity'] = weatherForecast['humidity'].values
    forecast = m.predict(future)
    #m.plot(forecast)

    forecast_mean = forecast['yhat'][-Horizon:];
    forecast_mean = round(forecast_mean); forecast_mean[forecast_mean<0] = 0;
    forecastVolume = pd.DataFrame(forecast_mean);
    #forecastVolume.rename(columns={'yhat': 'Predicted'})
    forecastVolume.index = testVolume.index
    forecastVolume=forecastVolume['yhat']

    forecast_ci= forecast[['yhat_lower','yhat_upper']][-Horizon:];
    forecast_ci = round(forecast_ci); forecast_ci[forecast_ci<0] = 0;
    pred_ci= pd.DataFrame(forecast_ci);
    pred_ci.index=testVolume.index
    yhat_lower = pred_ci['yhat_lower']; yhat_upper = pred_ci['yhat_upper'];

    ## Loading ARIMA model from disk
    #filename = 'finalized_model.sav'
    #results = pickle.load(open(filename, 'rb'))
    #
    #pred = results.get_prediction(start = testStart, end = testEnd)
    #pred_ci = pred.conf_int();
    #forecastVolume = pred.predicted_mean;
    #forecastVolume = round(forecastVolume); forecastVolume[forecastVolume<0] = 0;
    
    histVolume = trainVolume[-2*Horizon:];
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=histVolume.index, y=histVolume, name= 'Historical'))
    fig.add_trace(go.Scatter(x=forecastVolume.index, y=forecastVolume, name= 'Forecast'))
    fig.add_trace(go.Scatter(x=testVolume.index, y=testVolume, name= 'Observed'))
    fig.add_trace(go.Scatter(x=yhat_lower.index, y=yhat_lower, 
                            mode='lines',
                            line=dict(width=0.1),
                            name=''))
    fig.add_trace(go.Scatter(x=yhat_upper.index, y=yhat_upper, fill = 'tonexty', 
                            mode='lines',
                            line=dict(width=0.1),
                            name='Confidence interval'))
    fig.update_xaxes(title_text='Time')
    if RoadUserID == 1: 
        fig.update_yaxes(title_text='Pedestrian volume')
    elif RoadUserID == 2:
        fig.update_yaxes(title_text='Cyclist volume')
    fig.update_layout(title_text = Intersection + ' Intersection')
    fig.show()



In [10]:
def get_maps(RoadUserID, IntID, testStart, testEnd, Horizon):
    #print("This may take a while...\n\n")
    
    if RoadUserID == 1:
        X = Pedestrians
    elif RoadUserID == 2:
        X = Cyclists

    TrainSize = 24*7*4;
    trainStart = testStart - pd.Timedelta(hours = TrainSize)
    trainEnd = testStart - pd.Timedelta(hours = 1)
    
    IntNames = ['Adelaide-Bay', 'Adelaide-Jarvis', 'Front-Bathurst', 'King-Portland',
               'King-Peter', 'King-Bay', 'King-Yonge', 'King-Church', 'King-Jarvis',
               'Richmond-Spadina','Richmond-Bay','Wellington-Blue_Jays',
                'Wellington-Bay','King-John','Queen-Bathurst'];
    
    IntLoc = Locations.iloc[0,:]
    trafficMap = folium.Map(location= IntLoc, zoom_start = 14)
    
    IntForecasts = [ [] for i in range(Horizon) ]
    for IntID in range(len(X)-1):
        #print(IntID)
        Peds = X[IntID];
        trainVolume = Peds[trainStart: trainEnd]['volume']
        
        testVolume = Peds[testStart: testEnd]['volume'];
        weatherForecast = Peds[trainStart: testEnd][['tempC','humidity']];

        pInput = Peds[trainStart: trainEnd][['dateTimeIdx','volume',
                     'tempC','humidity']]
        pInput = pInput.rename(columns={"dateTimeIdx": "ds", "volume": "y"})
        
        #Using Prophet
        m = Prophet(yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=True);
        m.add_regressor('tempC')
        m.add_regressor('humidity')
        m.fit(pInput)
        future = m.make_future_dataframe(periods = Horizon, freq = 'h');
        future['tempC'] = weatherForecast['tempC'].values
        future['humidity'] = weatherForecast['humidity'].values
        forecast = m.predict(future)
        #m.plot(forecast)

        forecast_mean = forecast['yhat'][-Horizon:];
        forecast_mean = round(forecast_mean); forecast_mean[forecast_mean<0] = 0;
        forecastVolume = pd.DataFrame(forecast_mean);
        #forecastVolume.rename(columns={'yhat': 'Predicted'})
        forecastVolume.index = testVolume.index
        forecastVolume=forecastVolume['yhat']

        forecast_ci= forecast[['yhat_lower','yhat_upper']][-Horizon:];
        forecast_ci = round(forecast_ci); forecast_ci[forecast_ci<0] = 0;
        pred_ci= pd.DataFrame(forecast_ci);
        pred_ci.index=testVolume.index

        ## Loading ARIMA model from disk
        #filename = 'finalized_model.sav'
        #results = pickle.load(open(filename, 'rb'))
        #
        #pred = results.get_prediction(start = testStart, end = testEnd)
        #pred_ci = pred.conf_int();
        #forecastVolume = pred.predicted_mean;
        #forecastVolume = round(forecastVolume); forecastVolume[forecastVolume<0] = 0;
        
        IntLoc = Locations.iloc[IntID,:]
        
        folium.Marker(location = IntLoc, popup = IntNames[IntID]).add_to(trafficMap) 
        
        for i in range(Horizon):
            IntForecasts[i].append([IntLoc.values[0],IntLoc.values[1],forecast_mean.values[i]])
    
    #print(IntForecasts)
    HeatMapWithTime(IntForecasts, radius=10, auto_play = True,
                    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
                    min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(trafficMap)
    display(trafficMap)

In [11]:
grid = GridspecLayout(1, 4, grid_gap="10px")

In [12]:
InterSection=widgets.Dropdown(
    options=['Adelaide-Bay', 'Adelaide-Jarvis','Front-Bathurst', 
             'King-Bay', 'King-Church', 'King-Jarvis',
             'King-John', 'King-Peter', 'King-Portland', 
             'King-Yonge', 'Queen-Bathurst', 'Richmond-Bay',
             'Richmond-Spadina', 'Wellington-Bay', 'Wellington-Blue_Jays'],
    description='Intersection:',
    value='Adelaide-Bay',
    disabled=False,
)
grid[0,1]= InterSection

In [13]:
RoadUser=widgets.Dropdown(
    options=['Pedestrians', 'Cyclists'],
    description='Road user:',
    value='Pedestrians',
    disabled=False,
)
grid[0,2] = RoadUser

In [14]:
display(grid)

GridspecLayout(children=(Dropdown(description='Intersection:', layout=Layout(grid_area='widget001'), options=(…

In [15]:
grid = GridspecLayout(1, 3, grid_gap="10px")

In [16]:
style = {'description_width': 'initial'}
ForecastDate=widgets.DatePicker(
    style=style,
    description='Forecast start date:',
    value= pd.to_datetime('2019-06-26'),
    disabled=False
)
grid[0,0] = ForecastDate

In [17]:
style = {'description_width': 'initial'}
StartHour=widgets.Dropdown(
    options=['0','1', '2', '3','4','5','6','7','8','9','11','12',
            '13','14','15','16','17','18','19','20','21','22','23','24'],
    style= style,
    description='Forecast start hour:',
    value='0',
    disabled=False,
)
grid[0,1] = StartHour

In [18]:
style = {'description_width': 'initial'}
forecastHorizon= widgets.IntText(
    value=24,
    style= style,
    description='Forecast horizon in hours:',
    disabled=False
)
grid[0,2] = forecastHorizon

In [19]:
display(grid)

GridspecLayout(children=(DatePicker(value=Timestamp('2019-06-26 00:00:00'), description='Forecast start date:'…

In [20]:
grid = GridspecLayout(2, 2, grid_gap="10px")

In [21]:
ts_button = widgets.Button(description="Click here for forecast")
ts_output = widgets.Output()
grid[0,0] = ts_button

def on_ts_button_clicked(b):
    #grid[1,:] = ts_output;
    with ts_output:
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon)        
        ts_output.clear_output(wait=True)
        #maps_output.clear_output(wait=True)
ts_button.on_click(on_ts_button_clicked)

In [22]:
maps_button = widgets.Button(description="Click here for map")
maps_output = widgets.Output()
grid[0,1] = maps_button

def on_maps_button_clicked(b):
    #grid[1,:] = maps_output;
    with maps_output:
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        get_maps(RoadUserID, testStart, testStart, testEnd, Horizon)
        #ts_output.clear_output(wait=True)
        maps_output.clear_output(wait=True)
maps_button.on_click(on_maps_button_clicked)

In [23]:
display(grid)
display(maps_output)
display(ts_output)

GridspecLayout(children=(Button(description='Click here for forecast', layout=Layout(grid_area='widget001'), s…

Output()

Output()